## Calculate a Loan Instalment Schedule
Given principle, number of instalment and interest rate (per instalment):

* Calculates the EMI (equal monthly instalment) amount 
* Calculates the instalment amortisation schedule
* Maps to a DBEI loan in Mambu with Days-in-year: 30/360
* In terms of calculating EMI performs same calculation as Excel PMT function



In [145]:
import sympy as sp
import sys
# Need to increase the recusion limit (else .subs fails for 300 instalments)
sys.setrecursionlimit(5000)

In [198]:
def schedule(numInstalments):
    P, r, n, E = sp.symbols('P r n E')
    instalmentList = [{'num':1,'interest_expected':P*r,'principle_remaining':P*(1+r)-E}]
    for i in range(1,numInstalments):
        instObj = {}
        instObj["num"] = i+1
        instObj["interest_expected"] = instalmentList[i-1]['principle_remaining'] * r
        instObj["principle_remaining"] = instalmentList[i-1]['principle_remaining']*(1+r)-E
        instalmentList.append(instObj)
    return instalmentList

In [199]:
def expandSchedule(OrigPrinciple, interestRatePerInstalment, numInstalments):
    ROUND_NUMDIGITS = 10
    shList = schedule(numInstalments)
    
    # Solve the Equation for E - for the last instalment
    prin = shList[numInstalments-1]['principle_remaining']
    expandedPrin = prin.subs(r,interestRatePerInstalment)
    expandedPrin = expandedPrin.subs(P,OrigPrinciple)
    equalMonthlyInstalment = sp.solve(expandedPrin, E)[0]
    
    shList2 = schedule(numInstalments)
    
    # Now gothrough the complete shList and plug in values for all the variables
    for i in range(numInstalments):
        instObj = shList2[i]
        instObj["interest_expected"] = round(instObj["interest_expected"].subs(
            {
                r:interestRatePerInstalment,
                P:OrigPrinciple,
                E:equalMonthlyInstalment
            }), ROUND_NUMDIGITS)
        instObj["principle_remaining"] = round(instObj["principle_remaining"].subs(
            {
                r:interestRatePerInstalment,
                P:OrigPrinciple,
                E:equalMonthlyInstalment
            }), ROUND_NUMDIGITS)
        
        if i == 0:
            previousPrinciple = OrigPrinciple
        else:
            previousPrinciple = shList2[i-1]["principle_remaining"]
        instObj["principle_expected"] = previousPrinciple - instObj["principle_remaining"]
        
        instObj["total_expected"] = instObj["principle_expected"] + instObj["interest_expected"]
    
    return (equalMonthlyInstalment,shList2)

In [202]:
expandSchedule(OrigPrinciple=5000, interestRatePerInstalment=0.01, numInstalments=5 )

(1030.19899807940,
 [{'num': 1,
   'interest_expected': 50.0000000000000,
   'principle_remaining': 4019.8010019206,
   'principle_expected': 980.19899807940,
   'total_expected': 1030.19899807940},
  {'num': 2,
   'interest_expected': 40.1980100192,
   'principle_remaining': 3029.8000138604,
   'principle_expected': 990.00098806020,
   'total_expected': 1030.1989980794},
  {'num': 3,
   'interest_expected': 30.2980001386,
   'principle_remaining': 2029.8990159196,
   'principle_expected': 999.90099794080,
   'total_expected': 1030.1989980794},
  {'num': 4,
   'interest_expected': 20.2989901592,
   'principle_remaining': 1019.9990079994,
   'principle_expected': 1009.9000079202,
   'total_expected': 1030.1989980794},
  {'num': 5,
   'interest_expected': 10.1999900800,
   'principle_remaining': 0.0,
   'principle_expected': 1019.99900799940,
   'total_expected': 1030.19899807940}])